#### Importar Bibliotecas

In [1]:
import osmnx as ox
import networkx as nx
import folium
import pandas as pd
import sklearn
from functools import lru_cache

#### Endereços do roteiro

In [7]:
enderecos = list(pd.read_excel(r'input/Endereços.xlsx')['Endereços'].unique())

In [10]:
enderecos

['Parque Villa-Lobos, Alto de Pinheiros, são paulo, Brazil',
 'Parque Povo, Vila Olimpia, São Paulo, Brazil',
 'Parque do ibirapuera, Moema, São Paulo, Brazil',
 'Estadio do Pacaembu, Pacaembu, São Paulo, Brazil',
 'Estadio do Morumbi, Morumbi, São Paulo, Brazil',
 'Estadio allianz parque, Barra Funda, São Paulo, Brazil',
 'Parque da agua branca, Barra Funda, São Paulo, Brazil']

#### Modelo de Otimização

In [9]:
# Função para calcular o comprimento total da rota
def calcular_comprimento(grafo, rota):
    comprimento = 0
    for i in range(len(rota) - 1):
        comprimento += nx.shortest_path_length(grafo, rota[i], rota[i+1], weight='length')
    return comprimento

# Encontrar a rota mais eficiente usando o Algoritmo do Vizinho Mais Próximo
def rota_vizinho_mais_proximo(grafo, nos_inicio):
    todos_nos = set(nos_inicio)
    rota = [nos_inicio[0]]
    while len(rota) < len(nos_inicio):
        atual = rota[-1]
        proximos_nos = todos_nos - set(rota)
        proximo = min(proximos_nos, key=lambda no: nx.shortest_path_length(grafo, atual, no, weight='length'))
        rota.append(proximo)
    return rota

# Especificar a localidade e limitar a área do grafo
localidade = 'São Paulo, Brazil'
distancia = 50000  # 50 km

# Obter o ponto central da localidade
centro_localidade = ox.geocode(localidade)

# Carregar a rede de ruas dentro da distância especificada a partir do ponto central
grafo = ox.graph_from_point(centro_localidade, dist=distancia, network_type='drive')

# Função de cache para geocodificação
@lru_cache(maxsize=None)
def geocode_cache(endereco):
    return ox.geocode(endereco)

# Converter endereços em nós no grafo e mapear nós para endereços
enderecos_nos = {}
for endereco in enderecos:
    localizacao = geocode_cache(endereco)
    no = ox.nearest_nodes(grafo, localizacao[1], localizacao[0])
    enderecos_nos[no] = endereco

# Encontrar a rota mais eficiente
nos_iniciais = list(enderecos_nos.keys())
rota_eficiente = rota_vizinho_mais_proximo(grafo, nos_iniciais)

# Calcular e mostrar a rota
rota = []
for i in range(len(rota_eficiente) - 1):
    caminho = nx.shortest_path(grafo, rota_eficiente[i], rota_eficiente[i + 1], weight='length')
    rota.extend(caminho if i == 0 else caminho[1:])

# Criar um DataFrame para a rota
dados_rota = []
for no in rota:
    ponto = grafo.nodes[no]
    dados_rota.append({
        "Nó": no,
        "Latitude": ponto['y'],
        "Longitude": ponto['x'],
        "Endereço": enderecos_nos.get(no, "N/A")
    })

df_rota = pd.DataFrame(dados_rota)

# Criar um mapa interativo usando Folium
m = ox.plot_route_folium(grafo, rota)

# Adicionar marcadores para cada nó com a sequência numérica
parada = 1
for no in rota_eficiente:
    if no in enderecos_nos:
        ponto = grafo.nodes[no]
        popup_text = f"No: {no}<br>Endereço: {enderecos_nos[no]}<br>Parada: {parada}"
        folium.Marker(
            location=(ponto['y'], ponto['x']),
            popup=folium.Popup(popup_text, max_width=300),
            icon=folium.Icon(color="red", icon="info-sign")
        ).add_to(m)
        parada += 1

# Salvar o dataframe com roterio
df_rota.loc[df_rota['Endereço']!='N/A'].reset_index(drop=True).to_excel(r'output/roteiro.xlsx', index=False)

# Salvar o mapa interativo
m.save('mapa_da_rota.html')

# Mostrar o mapa
m

/var/folders/wy/4_8wnj8d5x9_cjytnn5333yh0000gn/T/ipykernel_38679/1964660323.py:71: FutureWarning: The `folium` module has been deprecated and will be removed in the v2.0.0 release. You can generate and explore interactive web maps of graph nodes, edges, and/or routes automatically using GeoPandas.GeoDataFrame.explore instead, for example like: `ox.graph_to_gdfs(G, nodes=False).explore()`. See the OSMnx examples gallery for complete details and demonstrations.
  m = ox.plot_route_folium(grafo, rota)
